In [2]:
import json
import pandas as pd
from urllib.parse import parse_qs, urlparse, unquote
from typing import Dict, List
from datetime import datetime

class LogAnalyzer:
    def __init__(self):
        self.logs = []
        self.parsed_logs = []

    def parse_log_file_from_path(self, file_path: str, output_txt: str = None, output_xlsx: str = None):
        """
        파일 경로에서 로그 파일을 읽어서 분석하고 결과 출력
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            
            # 로그 파싱
            self._parse_log_content(content)
            
            # txt 파일 출력
            if output_txt:
                self._export_to_txt(output_txt)
            
            # Excel 파일 출력
            if output_xlsx:
                self._export_to_excel(output_xlsx)
                
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        except Exception as e:
            print(f"❌ 파일 읽기 오류: {e}")

    def _parse_log_content(self, content: str):
        """로그 내용 파싱"""
        self.logs = []
        self.parsed_logs = []
        lines = content.strip().split('\n')
        
        for i, line in enumerate(lines):
            if line.strip():
                try:
                    log_data = json.loads(line.strip())
                    self.logs.append((i+1, log_data))
                except json.JSONDecodeError as e:
                    print(f"JSON 파싱 오류 (라인 {i+1}): {e}")
        
        print(f"=== 총 {len(self.logs)}개의 로그 발견 ===\n")
        
        # 각 로그의 REQUEST 파라미터 분석
        for log_num, log_data in self.logs:
            if 'REQUEST' in log_data:
                params = self._parse_request_url(log_data['REQUEST'])
                if params:
                    parsed_log = {
                        'log_number': log_num,
                        'timestamp': log_data.get('LOGTIME', 'N/A'),
                        'params': params
                    }
                    self.parsed_logs.append(parsed_log)

    def _parse_request_url(self, request_url: str) -> Dict[str, str]:
        """REQUEST URL에서 파라미터를 파싱하고 한글을 올바르게 처리"""
        try:
            # 유니코드 이스케이프 디코딩
            url = request_url.encode('utf-8').decode('unicode_escape')
            parsed_url = urlparse(url)
            query_params = parse_qs(parsed_url.query, keep_blank_values=True)
            
            processed_params = {}
            for key, value_list in query_params.items():
                raw_value = value_list[0] if value_list else ''
                decoded_value = self._fix_korean_text(raw_value)
                processed_params[key] = decoded_value
            
            return processed_params
            
        except Exception as e:
            print(f"파싱 오류: {e}")
            return {}

    def _fix_korean_text(self, text: str) -> str:
        """깨진 한글을 올바르게 디코딩"""
        try:
            decoded = unquote(text, encoding='utf-8')
            
            if self._has_broken_korean(decoded):
                try:
                    fixed = decoded.encode('latin-1').decode('utf-8')
                    return fixed
                except:
                    try:
                        byte_data = bytes([ord(c) for c in decoded if ord(c) < 256])
                        fixed = byte_data.decode('utf-8', errors='ignore')
                        return fixed
                    except:
                        pass
            
            return decoded
            
        except Exception:
            return text

    def _has_broken_korean(self, text: str) -> bool:
        """깨진 한글 패턴이 있는지 확인"""
        broken_patterns = [
            'ë¶', 'ìë', 'ê¼¬', 'ë', 'í¼', 'ì¸', 'ê¸°', 'ê¸', 'ìì¹', 'ì', 'ê'
        ]
        return any(pattern in text for pattern in broken_patterns)

    def _export_to_txt(self, output_file: str):
        """파싱 결과를 txt 파일로 출력"""
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(f"=== 총 {len(self.logs)}개의 로그 발견 ===\n\n")
                
                for parsed_log in self.parsed_logs:
                    f.write("=" * 60 + "\n")
                    f.write(f"로그 #{parsed_log['log_number']} - {parsed_log['timestamp']}\n")
                    f.write("=" * 60 + "\n")
                    
                    params = parsed_log['params']
                    if params:
                        # URL 정보 (첫 번째 로그에서 추출)
                        for log_num, log_data in self.logs:
                            if log_num == parsed_log['log_number']:
                                if 'REQUEST' in log_data:
                                    url = log_data['REQUEST'].encode('utf-8').decode('unicode_escape')
                                    parsed_url = urlparse(url)
                                    f.write(f"URL: {parsed_url.netloc}{parsed_url.path}\n")
                                    f.write(f"총 파라미터 수: {len(params)}\n")
                                    f.write("-" * 50 + "\n")
                                break
                        
                        # 파라미터 출력
                        for key, value in params.items():
                            f.write(f"{key:20} : {value}\n")
                    else:
                        f.write("REQUEST 필드가 없습니다.\n")
                    
                    f.write("\n")
            
            print(f"✅ txt 파일 저장: {output_file}")
            
        except Exception as e:
            print(f"❌ txt 파일 저장 오류: {e}")

    def _export_to_excel(self, output_file: str):
        """파싱 결과를 Excel 파일로 출력 (page_id, click_type, act_type, click_text, key_list, value_list, len(key_list))"""
        try:
            excel_data = []
            
            for parsed_log in self.parsed_logs:
                params = parsed_log['params']
                
                # 각 로그에서 필요한 정보 추출
                page_id = params.get('page_id', '')
                click_type = params.get('click_type', '')
                act_type = params.get('act_type', '')
                click_text = params.get('click_text', '')
                
                # key_list와 value_list 생성
                key_list = list(params.keys())
                value_list = list(params.values())
                
                key_list_str = ', '.join(key_list)
                value_list_str = ', '.join(str(v) for v in value_list)
                key_count = len(key_list)
                
                excel_data.append({
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'click_text': click_text,
                    'key_list': key_list_str,
                    'value_list': value_list_str,
                    'len(key_list)': key_count
                })
            
            # DataFrame 생성 및 Excel 저장
            df = pd.DataFrame(excel_data)
            df.to_excel(output_file, index=False, engine='openpyxl')
            
            print(f"✅ Excel 파일 저장: {output_file}")
            print(f"   총 {len(excel_data)}개 로그 처리됨")
            
        except Exception as e:
            print(f"❌ Excel 파일 저장 오류: {e}")

    def analyze_logs(self, file_path: str):
        """로그 분석 (기존 기능 유지)"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            self._parse_log_content(content)
            
            # 콘솔 출력
            for parsed_log in self.parsed_logs:
                print("=" * 60)
                print(f"로그 #{parsed_log['log_number']} - {parsed_log['timestamp']}")
                print("=" * 60)
                
                params = parsed_log['params']
                if params:
                    # URL 정보 출력
                    for log_num, log_data in self.logs:
                        if log_num == parsed_log['log_number']:
                            if 'REQUEST' in log_data:
                                url = log_data['REQUEST'].encode('utf-8').decode('unicode_escape')
                                parsed_url = urlparse(url)
                                print(f"URL: {parsed_url.netloc}{parsed_url.path}")
                                print(f"총 파라미터 수: {len(params)}")
                                print("-" * 50)
                            break
                    
                    # 파라미터 출력
                    for key, value in params.items():
                        print(f"{key:20} : {value}")
                else:
                    print("REQUEST 필드가 없습니다.")
                
                print()
                
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        except Exception as e:
            print(f"❌ 파일 읽기 오류: {e}")

# 사용 예시
def run_example():
    """사용 예시"""
    analyzer = LogAnalyzer()
    
    # 1. 로그 분석 및 txt, Excel 파일 생성
    analyzer.parse_log_file_from_path(
        file_path="log_file.txt",
        output_txt="./result/log_analysis_result.txt",
        output_xlsx="./result/log_analysis_result.xlsx"
    )
    
    # 2. 콘솔에 로그 분석 결과 출력만 하고 싶은 경우
    # analyzer.analyze_logs("log_file.txt")

# 기존 QA 검증 클래스 (기존 기능 유지)
class LogQAValidator:
    def __init__(self):
        self.scenarios = []
        self.logs = []
        self.unique_keys = []
        
    def load_scenario_csv(self, csv_file_path: str):
        """CSV 파일에서 시나리오 로드"""
        self.scenarios = []
        df = pd.read_csv(csv_file_path, encoding='utf-8')
        
        # 컬럼 구성에 따라 고유키 조합 결정
        columns = df.columns.tolist()
        if 'page_id' in columns:
            self.unique_keys = ['page_id', 'click_type', 'act_type']
            print("🔑 고유키 조합: page_id + click_type + act_type")
        elif 'click_text' in columns:
            self.unique_keys = ['click_text', 'click_type', 'act_type']
            print("🔑 고유키 조합: click_text + click_type + act_type")
        else:
            raise ValueError("지원하지 않는 시나리오 형식입니다. page_id 또는 click_text 컬럼이 필요합니다.")
        
        for _, row in df.iterrows():
            key_list = [k.strip() for k in str(row['key_list']).split(',') if k.strip()]
            scenario = {
                'expected_keys': key_list
            }
            
            # 고유키 값들 추가
            for key in self.unique_keys:
                scenario[key] = str(row[key]) if pd.notna(row[key]) else ''
            
            self.scenarios.append(scenario)
        
        print(f"✅ 시나리오 로드: {len(self.scenarios)}개")

    def parse_logs_from_file(self, log_file_path: str):
        """로그 txt 파일에서 직접 파싱"""
        try:
            with open(log_file_path, 'r', encoding='utf-8') as file:
                log_content = file.read()
            self.parse_logs(log_content)
            print(f"✅ 로그 파일 로드: {log_file_path}")
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {log_file_path}")
        except Exception as e:
            print(f"❌ 파일 읽기 오류: {e}")

    def parse_logs(self, log_content: str):
        """로그 파싱"""
        self.logs = []
        for i, line in enumerate(log_content.strip().split('\n')):
            if not line.strip():
                continue
                
            try:
                log_data = json.loads(line.strip())
                if 'REQUEST' not in log_data:
                    continue
                    
                params = self._parse_url(log_data['REQUEST'])
                if params:
                    self.logs.append({
                        'line': i + 1,
                        'timestamp': log_data.get('LOGTIME', ''),
                        'params': params
                    })
            except json.JSONDecodeError:
                continue
                
        print(f"✅ 로그 파싱: {len(self.logs)}개")

    def _parse_url(self, request_url: str) -> Dict[str, str]:
        """URL 파라미터 파싱"""
        try:
            url = request_url.encode('utf-8').decode('unicode_escape')
            parsed = urlparse(url)
            query_params = parse_qs(parsed.query, keep_blank_values=True)
            
            result = {}
            for key, values in query_params.items():
                if values:
                    decoded = unquote(values[0], encoding='utf-8')
                    # 한글 깨짐 복구 시도
                    try:
                        if any(ord(c) > 127 for c in decoded):
                            decoded = decoded.encode('latin-1').decode('utf-8')
                    except:
                        pass
                    result[key] = decoded
            return result
        except:
            return {}

    def validate_and_export(self, output_file: str):
        """검증 수행 및 결과 XLSX 출력"""
        results = []
        
        for scenario in self.scenarios:
            # 시나리오와 매칭되는 로그 찾기
            matched_logs = self._find_matching_logs(scenario)
            
            if not matched_logs:
                # 매칭되는 로그가 없는 경우
                for key in scenario['expected_keys']:
                    result = {
                        'key': key,
                        'value': 'NOT_FOUND',
                        'pass': 'FAIL'
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)
                continue
            
            # 각 매칭된 로그에 대해 검증
            for log in matched_logs:
                actual_keys = set(log['params'].keys())
                expected_keys = set(scenario['expected_keys'])
                
                # 예상 키들 검증
                for key in expected_keys:
                    value = log['params'].get(key, 'MISSING')
                    pass_status = 'PASS' if key in actual_keys else 'FAIL'
                    
                    result = {
                        'key': key,
                        'value': value,
                        'pass': pass_status
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)
                
                # 예상치 못한 키들 (선택사항)
                unexpected_keys = actual_keys - expected_keys
                for key in unexpected_keys:
                    result = {
                        'key': key,
                        'value': log['params'][key],
                        'pass': 'UNEXPECTED'
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)

        # XLSX 출력 - 컬럼 순서 정렬
        df_results = pd.DataFrame(results)
        
        # 고유키 조합에 따른 컬럼 순서 정의
        if 'page_id' in self.unique_keys:
            column_order = ['page_id', 'click_type', 'act_type', 'key', 'value', 'pass']
        else:  # click_text 기반
            column_order = ['click_text', 'click_type', 'act_type', 'key', 'value', 'pass']
        
        # 컬럼 순서 적용
        df_results = df_results[column_order]
        df_results.to_excel(output_file, index=False, engine='openpyxl')
        
        # 요약 출력
        self._print_summary(results)
        print(f"✅ 결과 저장: {output_file}")

    def _find_matching_logs(self, scenario: Dict) -> List[Dict]:
        """시나리오와 매칭되는 로그 찾기"""
        matched = []
        
        for log in self.logs:
            params = log['params']
            
            # 동적 고유키로 매칭
            is_match = True
            for key in self.unique_keys:
                if params.get(key, '') != scenario.get(key, ''):
                    is_match = False
                    break
            
            if is_match:
                matched.append(log)
        
        return matched

    def _print_summary(self, results: List[Dict]):
        """결과 요약 출력"""
        df = pd.DataFrame(results)
        
        total = len(df)
        passed = len(df[df['pass'] == 'PASS'])
        failed = len(df[df['pass'] == 'FAIL'])
        unexpected = len(df[df['pass'] == 'UNEXPECTED'])
        
        print(f"\n📊 검증 결과:")
        print(f"   전체: {total}개")
        print(f"   통과: {passed}개")
        print(f"   실패: {failed}개")
        print(f"   예상외: {unexpected}개")
        print(f"   성공률: {passed/total*100:.1f}%" if total > 0 else "   성공률: 0%")

if __name__ == "__main__":
    # 로그 분석 및 파일 출력 예시
    print("=== 로그 분석 및 파일 출력 ===")
    run_example()

=== 로그 분석 및 파일 출력 ===
JSON 파싱 오류 (라인 94): Expecting ',' delimiter: line 1 column 618 (char 617)
=== 총 385개의 로그 발견 ===

✅ txt 파일 저장: ./result/log_analysis_result.txt
✅ Excel 파일 저장: ./result/log_analysis_result.xlsx
   총 385개 로그 처리됨


### 기본 모델

In [16]:
# import json
# import pandas as pd
# from urllib.parse import parse_qs, urlparse, unquote
# from typing import Dict, List
# from datetime import datetime

# class LogAnalyzer:
#     def __init__(self):
#         self.logs = []
#         self.parsed_logs = []

#     def parse_log_file_from_path(self, file_path: str, output_txt: str = None, output_xlsx: str = None):
#         """
#         파일 경로에서 로그 파일을 읽어서 분석하고 결과 출력
#         """
#         try:
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 content = file.read()
            
#             # 로그 파싱
#             self._parse_log_content(content)
            
#             # txt 파일 출력
#             if output_txt:
#                 self._export_to_txt(output_txt)
            
#             # Excel 파일 출력
#             if output_xlsx:
#                 self._export_to_excel(output_xlsx)
                
#         except FileNotFoundError:
#             print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
#         except Exception as e:
#             print(f"❌ 파일 읽기 오류: {e}")

#     def _parse_log_content(self, content: str):
#         """로그 내용 파싱 - 안전한 JSON 파싱"""
#         self.logs = []
#         self.parsed_logs = []
#         lines = content.strip().split('\n')
        
#         json_errors = []
        
#         for i, line in enumerate(lines):
#             if line.strip():
#                 try:
#                     log_data = json.loads(line.strip())
#                     self.logs.append((i+1, log_data))
#                 except json.JSONDecodeError as e:
#                     json_errors.append((i+1, str(e), line.strip()))
                    
#                     # JSON 수정 시도
#                     fixed_line = self._try_fix_json(line.strip())
#                     if fixed_line:
#                         try:
#                             log_data = json.loads(fixed_line)
#                             self.logs.append((i+1, log_data))
#                             print(f"✅ JSON 수정 성공 (라인 {i+1})")
#                         except:
#                             pass
        
#         # 오류 상세 출력
#         if json_errors:
#             print(f"⚠️ JSON 파싱 오류 발생: {len(json_errors)}개")
#             print("="*80)
#             for line_num, error_msg, original_line in json_errors:
#                 print(f"라인 {line_num}: {error_msg}")
#                 print(f"원본 로그: {original_line[:200]}..." if len(original_line) > 200 else f"원본 로그: {original_line}")
#                 print("-" * 80)
#             print(f"   성공적으로 파싱된 로그: {len(self.logs)}개")
#             print(f"   전체 라인 수: {len(lines)}개")
#         else:
#             print(f"✅ 모든 JSON 파싱 성공")
        
#         print(f"=== 총 {len(self.logs)}개의 로그 처리 ===\n")
        
#         # 각 로그의 REQUEST 파라미터 분석
#         for log_num, log_data in self.logs:
#             if 'REQUEST' in log_data:
#                 params = self._parse_request_url(log_data['REQUEST'])
#                 if params:
#                     parsed_log = {
#                         'log_number': log_num,
#                         'timestamp': log_data.get('LOGTIME', 'N/A'),
#                         'params': params
#                     }
#                     self.parsed_logs.append(parsed_log)

#     def _try_fix_json(self, line: str) -> str:
#         """JSON 수정 시도 - 더 강력한 수정"""
#         try:
#             # 1. 마지막 쉼표 제거
#             if line.endswith(',}'):
#                 line = line[:-2] + '}'
#             elif line.endswith(',]'):
#                 line = line[:-2] + ']'
            
#             # 2. 값 내부의 따옴표 이스케이프 처리
#             import re
            
#             # JSON 키:값 패턴을 찾아서 값 내부의 따옴표를 이스케이프
#             def fix_quotes_in_values(match):
#                 key = match.group(1)
#                 value = match.group(2)
                
#                 # 값 내부의 따옴표를 이스케이프
#                 # 단, 이미 이스케이프된 따옴표는 제외
#                 fixed_value = value
                
#                 # 따옴표 앞에 백슬래시가 없는 경우만 이스케이프
#                 fixed_value = re.sub(r'(?<!\\)"', r'\\"', fixed_value)
                
#                 return f'"{key}":"{fixed_value}"'
            
#             # "key":"value" 패턴을 찾아서 수정
#             # 탐욕적이지 않은 매칭 사용
#             pattern = r'"([^"]+)":"(.*?)(?="[,}]|$)'
            
#             # 특별히 문제가 되는 패턴들을 먼저 처리
#             # click_text, article_title 등의 값에서 따옴표 문제 해결
#             problem_fields = ['click_text', 'article_title', 'banner_text']
            
#             for field in problem_fields:
#                 # 해당 필드의 값에서 따옴표 문제 수정
#                 field_pattern = f'"{field}":"([^"]*"[^"]*)"'
                
#                 def fix_field_quotes(match):
#                     value = match.group(1)
#                     # 값 내부의 따옴표를 이스케이프
#                     fixed_value = value.replace('"', '\\"')
#                     return f'"{field}":"{fixed_value}"'
                
#                 line = re.sub(field_pattern, fix_field_quotes, line)
            
#             # 3. 잘못된 이스케이프 문자 수정
#             line = line.replace('\\u003d', '=').replace('\\u0026', '&')
            
#             return line
            
#         except Exception as e:
#             # 더 간단한 수정 시도
#             try:
#                 # 값 내부의 모든 따옴표를 작은따옴표로 변경 (임시 해결책)
#                 import re
                
#                 # "key":"value with "quotes"" 패턴을 찾아서 수정
#                 def replace_inner_quotes(match):
#                     key = match.group(1)
#                     value = match.group(2)
#                     # 값 내부의 따옴표를 작은따옴표로 변경
#                     fixed_value = value.replace('"', "'")
#                     return f'"{key}":"{fixed_value}"'
                
#                 # 문제가 되는 필드들에 대해서만 적용
#                 problem_fields = ['click_text', 'article_title', 'banner_text']
#                 for field in problem_fields:
#                     pattern = f'"{field}":"([^"]*"[^"]*)"'
#                     line = re.sub(pattern, replace_inner_quotes, line)
                
#                 return line
#             except:
#                 return None

#     def _parse_request_url(self, request_url: str) -> Dict[str, str]:
#         """REQUEST URL에서 파라미터를 파싱하고 한글을 올바르게 처리"""
#         try:
#             # 유니코드 이스케이프 디코딩
#             url = request_url.encode('utf-8').decode('unicode_escape')
#             parsed_url = urlparse(url)
            
#             # 쿼리 파라미터를 수동으로 파싱 (복잡한 값 처리)
#             query = parsed_url.query
#             processed_params = {}
            
#             if not query:
#                 return processed_params
            
#             # &로 분할하되, URL 인코딩된 &(%26)은 분할하지 않음
#             params = []
#             current_param = ""
#             i = 0
            
#             while i < len(query):
#                 if query[i:i+1] == '&' and not query[max(0, i-3):i] == '%26':
#                     if current_param:
#                         params.append(current_param)
#                     current_param = ""
#                 else:
#                     current_param += query[i]
#                 i += 1
            
#             if current_param:
#                 params.append(current_param)
            
#             # 각 파라미터를 키=값으로 분할
#             for param in params:
#                 if '=' not in param:
#                     continue
                    
#                 # 첫 번째 = 기준으로만 분할 (값에 =이 포함될 수 있음)
#                 key, value = param.split('=', 1)
                
#                 # 키와 값 디코딩
#                 decoded_key = self._fix_korean_text(key)
#                 decoded_value = self._fix_korean_text(value)
                
#                 processed_params[decoded_key] = decoded_value
            
#             return processed_params
            
#         except Exception as e:
#             print(f"파싱 오류: {e}")
#             # 실패 시 기존 방식으로 fallback
#             try:
#                 url = request_url.encode('utf-8').decode('unicode_escape')
#                 parsed_url = urlparse(url)
#                 query_params = parse_qs(parsed_url.query, keep_blank_values=True)
                
#                 processed_params = {}
#                 for key, value_list in query_params.items():
#                     raw_value = value_list[0] if value_list else ''
#                     decoded_value = self._fix_korean_text(raw_value)
#                     processed_params[key] = decoded_value
                
#                 return processed_params
#             except:
#                 return {}

#     def _fix_korean_text(self, text: str) -> str:
#         """깨진 한글을 올바르게 디코딩"""
#         try:
#             # URL 디코딩 먼저 시도
#             decoded = unquote(text, encoding='utf-8')
            
#             # 깨진 한글 패턴이 있는지 확인
#             if self._has_broken_korean(decoded):
#                 # Latin-1로 인코딩 후 UTF-8로 디코딩 (일반적인 mojibake 해결법)
#                 try:
#                     fixed = decoded.encode('latin-1').decode('utf-8')
#                     return fixed
#                 except:
#                     pass
                    
#                 # 그래도 안되면 바이트 단위로 처리
#                 try:
#                     # 문자열의 각 문자를 바이트로 변환 후 UTF-8 디코딩
#                     byte_data = bytes([ord(c) for c in decoded if ord(c) < 256])
#                     fixed = byte_data.decode('utf-8', errors='ignore')
#                     return fixed
#                 except:
#                     pass
                    
#                 # UTF-8 바이트 패턴으로 복구 시도
#                 try:
#                     # 특정 깨진 패턴들을 직접 매핑
#                     broken_patterns = {
#                         'ê·¹ììëª¨': '극장용모',
#                         'í¥ê¸°ë¡­ê²': '향기롭게',
#                         'ì¤ê¸°ì¼ì´': '수기일이',
#                         'ììëª¨': '용모',
#                         'ê·¹': '극'
#                     }
                    
#                     result = decoded
#                     for broken, fixed in broken_patterns.items():
#                         result = result.replace(broken, fixed)
                    
#                     if result != decoded:
#                         return result
                        
#                 except:
#                     pass
            
#             return decoded
            
#         except Exception:
#             return text

#     def _has_broken_korean(self, text: str) -> bool:
#         """깨진 한글 패턴이 있는지 확인"""
#         broken_patterns = [
#             'ë¶', 'ìë', 'ê¼¬', 'ë', 'í¼', 'ì¸', 'ê¸°', 'ê¸', 'ìì¹', 'ì', 'ê',
#             'ê·¹ì', 'í¥ê¸°', 'ì¤ê¸°', 'ììë', 'ëª¨'
#         ]
#         return any(pattern in text for pattern in broken_patterns)

#     def _export_to_txt(self, output_file: str):
#         """파싱 결과를 txt 파일로 출력"""
#         try:
#             with open(output_file, 'w', encoding='utf-8') as f:
#                 f.write(f"=== 총 {len(self.logs)}개의 로그 발견 ===\n\n")
                
#                 for parsed_log in self.parsed_logs:
#                     f.write("=" * 60 + "\n")
#                     f.write(f"로그 #{parsed_log['log_number']} - {parsed_log['timestamp']}\n")
#                     f.write("=" * 60 + "\n")
                    
#                     params = parsed_log['params']
#                     if params:
#                         # URL 정보 (첫 번째 로그에서 추출)
#                         for log_num, log_data in self.logs:
#                             if log_num == parsed_log['log_number']:
#                                 if 'REQUEST' in log_data:
#                                     url = log_data['REQUEST'].encode('utf-8').decode('unicode_escape')
#                                     parsed_url = urlparse(url)
#                                     f.write(f"URL: {parsed_url.netloc}{parsed_url.path}\n")
#                                     f.write(f"총 파라미터 수: {len(params)}\n")
#                                     f.write("-" * 50 + "\n")
#                                 break
                        
#                         # 파라미터 출력
#                         for key, value in params.items():
#                             f.write(f"{key:20} : {value}\n")
#                     else:
#                         f.write("REQUEST 필드가 없습니다.\n")
                    
#                     f.write("\n")
            
#             print(f"✅ txt 파일 저장: {output_file}")
            
#         except Exception as e:
#             print(f"❌ txt 파일 저장 오류: {e}")

#     def _export_to_excel(self, output_file: str):
#         """파싱 결과를 Excel 파일로 출력 (page_id, click_type, act_type, click_text, key_list, value_list, len(key_list))"""
#         try:
#             excel_data = []
            
#             for parsed_log in self.parsed_logs:
#                 params = parsed_log['params']
                
#                 # 각 로그에서 필요한 정보 추출
#                 page_id = params.get('page_id', '')
#                 click_type = params.get('click_type', '')
#                 act_type = params.get('act_type', '')
#                 click_text = params.get('click_text', '')
                
#                 # key_list와 value_list 생성
#                 key_list = list(params.keys())
#                 value_list = [self._fix_korean_text(str(v)) for v in params.values()]  # value도 한글 디코딩 적용
                
#                 key_list_str = ', '.join(key_list)
#                 value_list_str = ', '.join(value_list)
#                 key_count = len(key_list)
                
#                 excel_data.append({
#                     'page_id': page_id,
#                     'click_type': click_type,
#                     'act_type': act_type,
#                     'click_text': click_text,
#                     'key_list': key_list_str,
#                     'value_list': value_list_str,
#                     'len(key_list)': key_count
#                 })
            
#             # DataFrame 생성 및 Excel 저장
#             df = pd.DataFrame(excel_data)
#             df.to_excel(output_file, index=False, engine='openpyxl')
            
#             print(f"✅ Excel 파일 저장: {output_file}")
#             print(f"   총 {len(excel_data)}개 로그 처리됨")
            
#         except Exception as e:
#             print(f"❌ Excel 파일 저장 오류: {e}")

#     def analyze_logs(self, file_path: str):
#         """로그 분석 (기존 기능 유지) - 오류 로그도 출력"""
#         try:
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 content = file.read()
            
#             # 오류 로그도 찾기 위해 별도 파싱
#             lines = content.strip().split('\n')
#             json_errors = []
            
#             for i, line in enumerate(lines):
#                 if line.strip():
#                     try:
#                         json.loads(line.strip())
#                     except json.JSONDecodeError as e:
#                         json_errors.append((i+1, str(e), line.strip()))
            
#             # 오류 로그 출력
#             if json_errors:
#                 print("="*80)
#                 print(f"🚨 JSON 파싱 오류 로그 상세 분석")
#                 print("="*80)
#                 for line_num, error_msg, original_line in json_errors:
#                     print(f"라인 {line_num}: {error_msg}")
#                     print(f"원본 로그: {original_line[:300]}..." if len(original_line) > 300 else f"원본 로그: {original_line}")
#                     print("-" * 80)
#                 print()
            
#             # 기존 파싱 로직 실행
#             self._parse_log_content(content)
            
#             # 콘솔 출력
#             for parsed_log in self.parsed_logs:
#                 print("=" * 60)
#                 print(f"로그 #{parsed_log['log_number']} - {parsed_log['timestamp']}")
#                 print("=" * 60)
                
#                 params = parsed_log['params']
#                 if params:
#                     # URL 정보 출력
#                     for log_num, log_data in self.logs:
#                         if log_num == parsed_log['log_number']:
#                             if 'REQUEST' in log_data:
#                                 url = log_data['REQUEST'].encode('utf-8').decode('unicode_escape')
#                                 parsed_url = urlparse(url)
#                                 print(f"URL: {parsed_url.netloc}{parsed_url.path}")
#                                 print(f"총 파라미터 수: {len(params)}")
#                                 print("-" * 50)
#                             break
                    
#                     # 파라미터 출력
#                     for key, value in params.items():
#                         print(f"{key:20} : {value}")
#                 else:
#                     print("REQUEST 필드가 없습니다.")
                
#                 print()
                
#         except FileNotFoundError:
#             print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
#         except Exception as e:
#             print(f"❌ 파일 읽기 오류: {e}")

# # 사용 예시
# def run_example():
#     """사용 예시"""
#     analyzer = LogAnalyzer()
    
#     # 1. 로그 분석 및 txt, Excel 파일 생성
#     analyzer.parse_log_file_from_path(
#         file_path="log_file.txt",
#         output_txt="log_analysis_result.txt",
#         output_xlsx="log_analysis_result.xlsx"
#     )
    
#     # 2. 콘솔에 로그 분석 결과 출력만 하고 싶은 경우
#     # analyzer.analyze_logs("log_file.txt")

# # 기존 QA 검증 클래스 (기존 기능 유지)
# class LogQAValidator:
#     def __init__(self):
#         self.scenarios = []
#         self.logs = []
#         self.unique_keys = []
        
#     def load_scenario_csv(self, csv_file_path: str):
#         """CSV 파일에서 시나리오 로드"""
#         self.scenarios = []
#         df = pd.read_csv(csv_file_path, encoding='utf-8')
        
#         # 컬럼 구성에 따라 고유키 조합 결정
#         columns = df.columns.tolist()
#         if 'page_id' in columns:
#             self.unique_keys = ['page_id', 'click_type', 'act_type']
#             print("🔑 고유키 조합: page_id + click_type + act_type")
#         elif 'click_text' in columns:
#             self.unique_keys = ['click_text', 'click_type', 'act_type']
#             print("🔑 고유키 조합: click_text + click_type + act_type")
#         else:
#             raise ValueError("지원하지 않는 시나리오 형식입니다. page_id 또는 click_text 컬럼이 필요합니다.")
        
#         for _, row in df.iterrows():
#             key_list = [k.strip() for k in str(row['key_list']).split(',') if k.strip()]
#             scenario = {
#                 'expected_keys': key_list
#             }
            
#             # 고유키 값들 추가
#             for key in self.unique_keys:
#                 scenario[key] = str(row[key]) if pd.notna(row[key]) else ''
            
#             self.scenarios.append(scenario)
        
#         print(f"✅ 시나리오 로드: {len(self.scenarios)}개")

#     def parse_logs_from_file(self, log_file_path: str):
#         """로그 txt 파일에서 직접 파싱"""
#         try:
#             with open(log_file_path, 'r', encoding='utf-8') as file:
#                 log_content = file.read()
#             self.parse_logs(log_content)
#             print(f"✅ 로그 파일 로드: {log_file_path}")
#         except FileNotFoundError:
#             print(f"❌ 파일을 찾을 수 없습니다: {log_file_path}")
#         except Exception as e:
#             print(f"❌ 파일 읽기 오류: {e}")

#     def parse_logs(self, log_content: str):
#         """로그 파싱 - 안전한 JSON 파싱"""
#         self.logs = []
#         lines = log_content.strip().split('\n')
        
#         json_errors = []
        
#         for i, line in enumerate(lines):
#             if not line.strip():
#                 continue
                
#             try:
#                 log_data = json.loads(line.strip())
#                 if 'REQUEST' not in log_data:
#                     continue
                    
#                 params = self._parse_url(log_data['REQUEST'])
#                 if params:
#                     self.logs.append({
#                         'line': i + 1,
#                         'timestamp': log_data.get('LOGTIME', ''),
#                         'params': params
#                     })
#             except json.JSONDecodeError as e:
#                 json_errors.append((i+1, str(e), line.strip()))
                
#                 # JSON 수정 시도
#                 fixed_line = self._try_fix_json_simple(line.strip())
#                 if fixed_line:
#                     try:
#                         log_data = json.loads(fixed_line)
#                         if 'REQUEST' in log_data:
#                             params = self._parse_url(log_data['REQUEST'])
#                             if params:
#                                 self.logs.append({
#                                     'line': i + 1,
#                                     'timestamp': log_data.get('LOGTIME', ''),
#                                     'params': params
#                                 })
#                                 print(f"✅ JSON 수정 성공 (라인 {i+1})")
#                     except:
#                         pass
        
#         # 오류 상세 출력
#         if json_errors:
#             print(f"⚠️ JSON 파싱 오류: {len(json_errors)}개")
#             print("="*80)
#             for line_num, error_msg, original_line in json_errors:
#                 print(f"라인 {line_num}: {error_msg}")
#                 print(f"원본 로그: {original_line[:200]}..." if len(original_line) > 200 else f"원본 로그: {original_line}")
#                 print("-" * 80)
#             print(f"✅ 성공적으로 파싱된 로그: {len(self.logs)}개")
#         else:
#             print(f"✅ 로그 파싱: {len(self.logs)}개")

#     def _try_fix_json_simple(self, line: str) -> str:
#         """간단한 JSON 수정 시도 - QA용"""
#         try:
#             # 1. 마지막 쉼표 제거
#             if line.endswith(',}'):
#                 line = line[:-2] + '}'
#             elif line.endswith(',]'):
#                 line = line[:-2] + ']'
            
#             # 2. 값 내부의 따옴표 문제 해결
#             import re
            
#             # 문제가 되는 필드들의 값에서 따옴표를 작은따옴표로 변경
#             problem_fields = ['click_text', 'article_title', 'banner_text']
            
#             for field in problem_fields:
#                 # "field":"value with "quotes"" 패턴 찾기
#                 pattern = f'"{field}":"([^"]*"[^"]*)"'
                
#                 def fix_quotes(match):
#                     value = match.group(1)
#                     # 값 내부의 따옴표를 작은따옴표로 변경
#                     fixed_value = value.replace('"', "'")
#                     return f'"{field}":"{fixed_value}"'
                
#                 line = re.sub(pattern, fix_quotes, line)
            
#             return line
            
#         except:
#             return None

#     def _parse_url(self, request_url: str) -> Dict[str, str]:
#         """URL 파라미터 파싱 (QA용)"""
#         try:
#             url = request_url.encode('utf-8').decode('unicode_escape')
#             parsed = urlparse(url)
            
#             # 동일한 개선된 파싱 로직 적용
#             query = parsed.query
#             result = {}
            
#             if not query:
#                 return result
            
#             # &로 분할하되, URL 인코딩된 &(%26)은 분할하지 않음
#             params = []
#             current_param = ""
#             i = 0
            
#             while i < len(query):
#                 if query[i:i+1] == '&' and not query[max(0, i-3):i] == '%26':
#                     if current_param:
#                         params.append(current_param)
#                     current_param = ""
#                 else:
#                     current_param += query[i]
#                 i += 1
            
#             if current_param:
#                 params.append(current_param)
            
#             # 각 파라미터를 키=값으로 분할
#             for param in params:
#                 if '=' not in param:
#                     continue
                    
#                 # 첫 번째 = 기준으로만 분할
#                 key, value = param.split('=', 1)
                
#                 # 키와 값 디코딩
#                 decoded_key = unquote(key, encoding='utf-8')
#                 decoded_value = unquote(value, encoding='utf-8')
                
#                 # 한글 깨짐 복구 시도
#                 try:
#                     if any(ord(c) > 127 for c in decoded_key):
#                         decoded_key = decoded_key.encode('latin-1').decode('utf-8')
#                     if any(ord(c) > 127 for c in decoded_value):
#                         decoded_value = decoded_value.encode('latin-1').decode('utf-8')
#                 except:
#                     pass
                    
#                 result[decoded_key] = decoded_value
            
#             return result
#         except:
#             # 실패 시 기존 방식으로 fallback
#             try:
#                 url = request_url.encode('utf-8').decode('unicode_escape')
#                 parsed = urlparse(url)
#                 query_params = parse_qs(parsed.query, keep_blank_values=True)
                
#                 result = {}
#                 for key, values in query_params.items():
#                     if values:
#                         decoded = unquote(values[0], encoding='utf-8')
#                         try:
#                             if any(ord(c) > 127 for c in decoded):
#                                 decoded = decoded.encode('latin-1').decode('utf-8')
#                         except:
#                             pass
#                         result[key] = decoded
#                 return result
#             except:
#                 return {}

#     def validate_and_export(self, output_file: str):
#         """검증 수행 및 결과 XLSX 출력"""
#         results = []
        
#         for scenario in self.scenarios:
#             # 시나리오와 매칭되는 로그 찾기
#             matched_logs = self._find_matching_logs(scenario)
            
#             if not matched_logs:
#                 # 매칭되는 로그가 없는 경우
#                 for key in scenario['expected_keys']:
#                     result = {
#                         'key': key,
#                         'value': 'NOT_FOUND',
#                         'pass': 'FAIL'
#                     }
                    
#                     # 고유키 값들 추가
#                     for uk in self.unique_keys:
#                         result[uk] = scenario.get(uk, '')
                    
#                     results.append(result)
#                 continue
            
#             # 각 매칭된 로그에 대해 검증
#             for log in matched_logs:
#                 actual_keys = set(log['params'].keys())
#                 expected_keys = set(scenario['expected_keys'])
                
#                 # 예상 키들 검증
#                 for key in expected_keys:
#                     value = log['params'].get(key, 'MISSING')
#                     pass_status = 'PASS' if key in actual_keys else 'FAIL'
                    
#                     result = {
#                         'key': key,
#                         'value': value,
#                         'pass': pass_status
#                     }
                    
#                     # 고유키 값들 추가
#                     for uk in self.unique_keys:
#                         result[uk] = scenario.get(uk, '')
                    
#                     results.append(result)
                
#                 # 예상치 못한 키들 (선택사항)
#                 unexpected_keys = actual_keys - expected_keys
#                 for key in unexpected_keys:
#                     result = {
#                         'key': key,
#                         'value': log['params'][key],
#                         'pass': 'UNEXPECTED'
#                     }
                    
#                     # 고유키 값들 추가
#                     for uk in self.unique_keys:
#                         result[uk] = scenario.get(uk, '')
                    
#                     results.append(result)

#         # XLSX 출력 - 컬럼 순서 정렬
#         df_results = pd.DataFrame(results)
        
#         # 고유키 조합에 따른 컬럼 순서 정의
#         if 'page_id' in self.unique_keys:
#             column_order = ['page_id', 'click_type', 'act_type', 'key', 'value', 'pass']
#         else:  # click_text 기반
#             column_order = ['click_text', 'click_type', 'act_type', 'key', 'value', 'pass']
        
#         # 컬럼 순서 적용
#         df_results = df_results[column_order]
#         df_results.to_excel(output_file, index=False, engine='openpyxl')
        
#         # 요약 출력
#         self._print_summary(results)
#         print(f"✅ 결과 저장: {output_file}")

#     def _find_matching_logs(self, scenario: Dict) -> List[Dict]:
#         """시나리오와 매칭되는 로그 찾기"""
#         matched = []
        
#         for log in self.logs:
#             params = log['params']
            
#             # 동적 고유키로 매칭
#             is_match = True
#             for key in self.unique_keys:
#                 if params.get(key, '') != scenario.get(key, ''):
#                     is_match = False
#                     break
            
#             if is_match:
#                 matched.append(log)
        
#         return matched

#     def _print_summary(self, results: List[Dict]):
#         """결과 요약 출력"""
#         df = pd.DataFrame(results)
        
#         total = len(df)
#         passed = len(df[df['pass'] == 'PASS'])
#         failed = len(df[df['pass'] == 'FAIL'])
#         unexpected = len(df[df['pass'] == 'UNEXPECTED'])
        
#         print(f"\n📊 검증 결과:")
#         print(f"   전체: {total}개")
#         print(f"   통과: {passed}개")
#         print(f"   실패: {failed}개")
#         print(f"   예상외: {unexpected}개")
#         print(f"   성공률: {passed/total*100:.1f}%" if total > 0 else "   성공률: 0%")

# if __name__ == "__main__":
#     # 로그 분석 및 파일 출력 예시
#     print("=== 로그 분석 및 파일 출력 ===")
#     run_example()

=== 로그 분석 및 파일 출력 ===
⚠️ JSON 파싱 오류 발생: 7개
라인 94: Expecting ',' delimiter: line 1 column 618 (char 617)
원본 로그: {"BYTES":"0","MODE":"","PROTOCOL":"HTTP/1.1","LOGTIME":"02/Jun/2025:18:26:13 +0900","REFERRER":"-","sid":"roundApp_web","USERAGENT":"Mozilla/5.0 (iPhone; CPU iPhone OS 18_5 like Mac OS X) AppleWebKit/...
--------------------------------------------------------------------------------
라인 422: Expecting ',' delimiter: line 1 column 453 (char 452)
원본 로그: {"BYTES":"0","MODE":"","PROTOCOL":"HTTP/1.1","LOGTIME":"04/Jun/2025:14:02:03 +0900","REFERRER":"-","sid":"roundApp_web","USERAGENT":"Mozilla/5.0 (iPhone; CPU iPhone OS 18_5 like Mac OS X) AppleWebKit/...
--------------------------------------------------------------------------------
라인 464: Expecting ',' delimiter: line 1 column 453 (char 452)
원본 로그: {"BYTES":"0","MODE":"","PROTOCOL":"HTTP/1.1","LOGTIME":"04/Jun/2025:14:04:04 +0900","REFERRER":"-","sid":"roundApp_web","USERAGENT":"Mozilla/5.0 (iPhone; CPU iPhone OS 18_5 like Mac

### 따옴표 강제 변환

In [69]:
import json
import pandas as pd
from urllib.parse import parse_qs, urlparse, unquote
from typing import Dict, List
from datetime import datetime

class LogAnalyzer:
    def __init__(self):
        self.logs = []
        self.parsed_logs = []

    def parse_log_file_from_path(self, file_path: str, output_txt: str = None, output_xlsx: str = None):
        """파일 경로에서 로그 파일을 읽어서 분석하고 결과 출력"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            
            # 로그 파싱
            self._parse_log_content(content)
            
            # txt 파일 출력
            if output_txt:
                self._export_to_txt(output_txt)
            
            # Excel 파일 출력
            if output_xlsx:
                self._export_to_excel(output_xlsx)
                
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        except Exception as e:
            print(f"❌ 파일 읽기 오류: {e}")

    def _parse_log_content(self, content: str):
        """로그 내용 파싱 - 안전한 JSON 파싱"""
        self.logs = []
        self.parsed_logs = []
        lines = content.strip().split('\n')
        
        json_errors = []
        
        for i, line in enumerate(lines):
            if line.strip():
                try:
                    log_data = json.loads(line.strip())
                    self.logs.append((i+1, log_data))
                except json.JSONDecodeError as e:
                    json_errors.append((i+1, str(e), line.strip()))
                    
                    # JSON 수정 시도
                    fixed_line = self._try_fix_json(line.strip())
                    if fixed_line:
                        try:
                            log_data = json.loads(fixed_line)
                            self.logs.append((i+1, log_data))
                            print(f"✅ JSON 수정 성공 (라인 {i+1})")
                        except:
                            # 추가 수정 시도
                            fixed_line2 = self._force_fix_json(line.strip())
                            if fixed_line2:
                                try:
                                    log_data = json.loads(fixed_line2)
                                    self.logs.append((i+1, log_data))
                                    print(f"✅ 강제 JSON 수정 성공 (라인 {i+1})")
                                except:
                                    pass
        
        # 오류 요약 출력
        if json_errors:
            print(f"⚠️ JSON 파싱 오류 발생: {len(json_errors)}개")
            successful_fixes = len(self.logs) - (len(lines) - len(json_errors))
            if successful_fixes > 0:
                print(f"✅ 수정 성공: {successful_fixes}개")
            print(f"✅ 최종 파싱된 로그: {len(self.logs)}개")
        else:
            print(f"✅ 모든 JSON 파싱 성공")
        
        print(f"=== 총 {len(self.logs)}개의 로그 처리 ===\n")
        
        # 각 로그의 REQUEST 파라미터 분석
        for log_num, log_data in self.logs:
            if 'REQUEST' in log_data:
                params = self._parse_request_url(log_data['REQUEST'])
                if params:
                    parsed_log = {
                        'log_number': log_num,
                        'timestamp': log_data.get('LOGTIME', 'N/A'),
                        'params': params
                    }
                    self.parsed_logs.append(parsed_log)

    def _try_fix_json(self, line: str) -> str:
        """JSON 수정 시도 - 1차 수정"""
        try:
            # 1. 마지막 쉼표 제거
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # 2. 값 내부의 따옴표를 작은따옴표로 변경
            import re
            
            # 모든 "key":"value with quotes" 패턴을 찾아서 수정
            def fix_quotes_in_value(match):
                key = match.group(1)
                value = match.group(2)
                # 값 내부의 모든 따옴표를 작은따옴표로 변경
                fixed_value = value.replace('"', "'")
                return f'"{key}":"{fixed_value}"'
            
            # 값에 따옴표가 포함된 패턴 찾기
            problem_pattern = r'"([^"]+)":"([^"]*"[^"]*)"'
            
            # 여러 번 적용하여 모든 문제 해결
            prev_line = ""
            iterations = 0
            while line != prev_line and iterations < 5:
                prev_line = line
                line = re.sub(problem_pattern, fix_quotes_in_value, line)
                iterations += 1
            
            return line
            
        except Exception:
            return None

    def _force_fix_json(self, line: str) -> str:
        """JSON 강제 수정 - 2차 수정"""
        try:
            # 기본 정리
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # JSON 구조 분석하여 키-값 쌍별로 처리
            if not (line.startswith('{') and line.endswith('}')):
                return None
            
            # 중괄호 제거
            content = line[1:-1]
            
            # 키-값 쌍들을 분리
            pairs = []
            current_pair = ""
            bracket_count = 0
            in_quotes = False
            escape_next = False
            
            for char in content:
                if escape_next:
                    current_pair += char
                    escape_next = False
                    continue
                
                if char == '\\':
                    current_pair += char
                    escape_next = True
                    continue
                
                if char == '"' and not escape_next:
                    in_quotes = not in_quotes
                
                if not in_quotes:
                    if char in '{}[]':
                        bracket_count += 1 if char in '{[' else -1
                    elif char == ',' and bracket_count == 0:
                        pairs.append(current_pair.strip())
                        current_pair = ""
                        continue
                
                current_pair += char
            
            if current_pair.strip():
                pairs.append(current_pair.strip())
            
            # 각 키-값 쌍을 수정
            fixed_pairs = []
            for pair in pairs:
                if ':' in pair:
                    try:
                        # 키와 값 분리 (첫 번째 콜론 기준)
                        colon_pos = pair.find(':')
                        key_part = pair[:colon_pos].strip()
                        value_part = pair[colon_pos+1:].strip()
                        
                        # 값 부분에서 따옴표로 둘러싸인 문자열인지 확인
                        if value_part.startswith('"') and value_part.endswith('"'):
                            # 값 내부의 따옴표를 작은따옴표로 변경
                            inner_value = value_part[1:-1]  # 양쪽 따옴표 제거
                            fixed_inner = inner_value.replace('"', "'")
                            fixed_value = f'"{fixed_inner}"'
                            fixed_pairs.append(f'{key_part}:{fixed_value}')
                        else:
                            fixed_pairs.append(pair)
                    except:
                        fixed_pairs.append(pair)
                else:
                    fixed_pairs.append(pair)
            
            # 다시 조합
            result = '{' + ','.join(fixed_pairs) + '}'
            return result
            
        except Exception:
            return None

    def _parse_request_url(self, request_url: str) -> Dict[str, str]:
        """REQUEST URL에서 파라미터를 파싱하고 한글을 올바르게 처리"""
        try:
            # 유니코드 이스케이프 디코딩
            url = request_url.encode('utf-8').decode('unicode_escape')
            parsed_url = urlparse(url)
            
            # 쿼리 파라미터를 수동으로 파싱 (복잡한 값 처리)
            query = parsed_url.query
            processed_params = {}
            
            if not query:
                return processed_params
            
            # &로 분할하되, URL 인코딩된 &(%26)은 분할하지 않음
            params = []
            current_param = ""
            i = 0
            
            while i < len(query):
                if query[i:i+1] == '&' and not query[max(0, i-3):i] == '%26':
                    if current_param:
                        params.append(current_param)
                    current_param = ""
                else:
                    current_param += query[i]
                i += 1
            
            if current_param:
                params.append(current_param)
            
            # 각 파라미터를 키=값으로 분할
            for param in params:
                if '=' not in param:
                    continue
                    
                # 첫 번째 = 기준으로만 분할 (값에 =이 포함될 수 있음)
                key, value = param.split('=', 1)
                
                # 키와 값 디코딩
                decoded_key = self._fix_korean_text(key)
                decoded_value = self._fix_korean_text(value)
                
                processed_params[decoded_key] = decoded_value
            
            return processed_params
            
        except Exception as e:
            print(f"파싱 오류: {e}")
            # 실패 시 기존 방식으로 fallback
            try:
                url = request_url.encode('utf-8').decode('unicode_escape')
                parsed_url = urlparse(url)
                query_params = parse_qs(parsed_url.query, keep_blank_values=True)
                
                processed_params = {}
                for key, value_list in query_params.items():
                    raw_value = value_list[0] if value_list else ''
                    decoded_value = self._fix_korean_text(raw_value)
                    processed_params[key] = decoded_value
                
                return processed_params
            except:
                return {}

    def _fix_korean_text(self, text: str) -> str:
        """깨진 한글을 올바르게 디코딩"""
        try:
            # URL 디코딩 먼저 시도
            decoded = unquote(text, encoding='utf-8')
            
            # 깨진 한글 패턴이 있는지 확인
            if self._has_broken_korean(decoded):
                # Latin-1로 인코딩 후 UTF-8로 디코딩 (일반적인 mojibake 해결법)
                try:
                    fixed = decoded.encode('latin-1').decode('utf-8')
                    return fixed
                except:
                    pass
                    
                # 그래도 안되면 바이트 단위로 처리
                try:
                    # 문자열의 각 문자를 바이트로 변환 후 UTF-8 디코딩
                    byte_data = bytes([ord(c) for c in decoded if ord(c) < 256])
                    fixed = byte_data.decode('utf-8', errors='ignore')
                    return fixed
                except:
                    pass
                    
                # UTF-8 바이트 패턴으로 복구 시도
                try:
                    # 특정 깨진 패턴들을 직접 매핑
                    broken_patterns = {
                        'ê·¹ììëª¨': '극장용모',
                        'í¥ê¸°ë¡­ê²': '향기롭게',
                        'ì¤ê¸°ì¼ì´': '수기일이',
                        'ììëª¨': '용모',
                        'ê·¹': '극'
                    }
                    
                    result = decoded
                    for broken, fixed in broken_patterns.items():
                        result = result.replace(broken, fixed)
                    
                    if result != decoded:
                        return result
                        
                except:
                    pass
            
            return decoded
            
        except Exception:
            return text

    def _has_broken_korean(self, text: str) -> bool:
        """깨진 한글 패턴이 있는지 확인"""
        broken_patterns = [
            'ë¶', 'ìë', 'ê¼¬', 'ë', 'í¼', 'ì¸', 'ê¸°', 'ê¸', 'ìì¹', 'ì', 'ê',
            'ê·¹ì', 'í¥ê¸°', 'ì¤ê¸°', 'ììë', 'ëª¨'
        ]
        return any(pattern in text for pattern in broken_patterns)

    def _export_to_txt(self, output_file: str):
        """파싱 결과를 txt 파일로 출력"""
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(f"=== 총 {len(self.logs)}개의 로그 발견 ===\n\n")
                
                for parsed_log in self.parsed_logs:
                    f.write("=" * 60 + "\n")
                    f.write(f"로그 #{parsed_log['log_number']} - {parsed_log['timestamp']}\n")
                    f.write("=" * 60 + "\n")
                    
                    params = parsed_log['params']
                    if params:
                        # URL 정보 (첫 번째 로그에서 추출)
                        for log_num, log_data in self.logs:
                            if log_num == parsed_log['log_number']:
                                if 'REQUEST' in log_data:
                                    url = log_data['REQUEST'].encode('utf-8').decode('unicode_escape')
                                    parsed_url = urlparse(url)
                                    f.write(f"URL: {parsed_url.netloc}{parsed_url.path}\n")
                                    f.write(f"총 파라미터 수: {len(params)}\n")
                                    f.write("-" * 50 + "\n")
                                break
                        
                        # 파라미터 출력
                        for key, value in params.items():
                            f.write(f"{key:20} : {value}\n")
                    else:
                        f.write("REQUEST 필드가 없습니다.\n")
                    
                    f.write("\n")
            
            print(f"✅ txt 파일 저장: {output_file}")
            
        except Exception as e:
            print(f"❌ txt 파일 저장 오류: {e}")

    def _export_to_excel(self, output_file: str):
        """파싱 결과를 Excel 파일로 출력"""
        try:
            excel_data = []
            
            for parsed_log in self.parsed_logs:
                params = parsed_log['params']
                
                # 각 로그에서 필요한 정보 추출
                page_id = params.get('page_id', '')
                click_type = params.get('click_type', '')
                act_type = params.get('act_type', '')
                click_text = params.get('click_text', '')
                
                # key_list와 value_list 생성
                key_list = list(params.keys())
                value_list = [self._fix_korean_text(str(v)) for v in params.values()]
                
                key_list_str = ', '.join(key_list)
                value_list_str = ', '.join(value_list)
                key_count = len(key_list)
                
                excel_data.append({
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'click_text': click_text,
                    'key_list': key_list_str,
                    'value_list': value_list_str,
                    'len(key_list)': key_count
                })
            
            # DataFrame 생성 및 Excel 저장
            df = pd.DataFrame(excel_data)
            df.to_excel(output_file, index=False, engine='openpyxl')
            
            print(f"✅ Excel 파일 저장: {output_file}")
            print(f"   총 {len(excel_data)}개 로그 처리됨")
            
        except Exception as e:
            print(f"❌ Excel 파일 저장 오류: {e}")

    def analyze_logs(self, file_path: str):
        """로그 분석 (콘솔 출력)"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            
            self._parse_log_content(content)
            
            # 콘솔 출력
            for parsed_log in self.parsed_logs:
                print("=" * 60)
                print(f"로그 #{parsed_log['log_number']} - {parsed_log['timestamp']}")
                print("=" * 60)
                
                params = parsed_log['params']
                if params:
                    # URL 정보 출력
                    for log_num, log_data in self.logs:
                        if log_num == parsed_log['log_number']:
                            if 'REQUEST' in log_data:
                                url = log_data['REQUEST'].encode('utf-8').decode('unicode_escape')
                                parsed_url = urlparse(url)
                                print(f"URL: {parsed_url.netloc}{parsed_url.path}")
                                print(f"총 파라미터 수: {len(params)}")
                                print("-" * 50)
                            break
                    
                    # 파라미터 출력
                    for key, value in params.items():
                        print(f"{key:20} : {value}")
                else:
                    print("REQUEST 필드가 없습니다.")
                
                print()
                
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        except Exception as e:
            print(f"❌ 파일 읽기 오류: {e}")


# QA 검증 클래스
class LogQAValidator:
    def __init__(self):
        self.scenarios = []
        self.logs = []
        self.unique_keys = []
        
    def load_scenario_csv(self, csv_file_path: str):
        """CSV 파일에서 시나리오 로드"""
        self.scenarios = []
        df = pd.read_csv(csv_file_path, encoding='utf-8')
        
        # 컬럼 구성에 따라 고유키 조합 결정
        columns = df.columns.tolist()
        if 'page_id' in columns:
            self.unique_keys = ['page_id', 'click_type', 'act_type']
            print("🔑 고유키 조합: page_id + click_type + act_type")
        elif 'click_text' in columns:
            self.unique_keys = ['click_text', 'click_type', 'act_type']
            print("🔑 고유키 조합: click_text + click_type + act_type")
        else:
            raise ValueError("지원하지 않는 시나리오 형식입니다. page_id 또는 click_text 컬럼이 필요합니다.")
        
        for _, row in df.iterrows():
            key_list = [k.strip() for k in str(row['key_list']).split(',') if k.strip()]
            scenario = {
                'expected_keys': key_list
            }
            
            # 고유키 값들 추가
            for key in self.unique_keys:
                scenario[key] = str(row[key]) if pd.notna(row[key]) else ''
            
            self.scenarios.append(scenario)
        
        print(f"✅ 시나리오 로드: {len(self.scenarios)}개")

    def parse_logs_from_file(self, log_file_path: str):
        """로그 txt 파일에서 직접 파싱"""
        try:
            with open(log_file_path, 'r', encoding='utf-8') as file:
                log_content = file.read()
            self.parse_logs(log_content)
            print(f"✅ 로그 파일 로드: {log_file_path}")
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {log_file_path}")
        except Exception as e:
            print(f"❌ 파일 읽기 오류: {e}")

    def parse_logs(self, log_content: str):
        """로그 파싱"""
        analyzer = LogAnalyzer()
        analyzer._parse_log_content(log_content)
        
        # LogAnalyzer의 결과를 QA용으로 변환
        self.logs = []
        for log_num, log_data in analyzer.logs:
            if 'REQUEST' in log_data:
                params = analyzer._parse_request_url(log_data['REQUEST'])
                if params:
                    self.logs.append({
                        'line': log_num,
                        'timestamp': log_data.get('LOGTIME', ''),
                        'params': params
                    })
        
        print(f"✅ QA용 로그 파싱: {len(self.logs)}개")

    def validate_and_export(self, output_file: str):
        """검증 수행 및 결과 XLSX 출력"""
        results = []
        
        for scenario in self.scenarios:
            # 시나리오와 매칭되는 로그 찾기
            matched_logs = self._find_matching_logs(scenario)
            
            if not matched_logs:
                # 매칭되는 로그가 없는 경우
                for key in scenario['expected_keys']:
                    result = {
                        'key': key,
                        'value': 'NOT_FOUND',
                        'pass': 'FAIL'
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)
                continue
            
            # 각 매칭된 로그에 대해 검증
            for log in matched_logs:
                actual_keys = set(log['params'].keys())
                expected_keys = set(scenario['expected_keys'])
                
                # 예상 키들 검증
                for key in expected_keys:
                    value = log['params'].get(key, 'MISSING')
                    pass_status = 'PASS' if key in actual_keys else 'FAIL'
                    
                    result = {
                        'key': key,
                        'value': value,
                        'pass': pass_status
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)
                
                # 예상치 못한 키들
                unexpected_keys = actual_keys - expected_keys
                for key in unexpected_keys:
                    result = {
                        'key': key,
                        'value': log['params'][key],
                        'pass': 'UNEXPECTED'
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)

        # XLSX 출력 - 컬럼 순서 정렬
        df_results = pd.DataFrame(results)
        
        # 고유키 조합에 따른 컬럼 순서 정의
        if 'page_id' in self.unique_keys:
            column_order = ['page_id', 'click_type', 'act_type', 'key', 'value', 'pass']
        else:
            column_order = ['click_text', 'click_type', 'act_type', 'key', 'value', 'pass']
        
        # 컬럼 순서 적용
        df_results = df_results[column_order]
        df_results.to_excel(output_file, index=False, engine='openpyxl')
        
        # 요약 출력
        self._print_summary(results)
        print(f"✅ 결과 저장: {output_file}")

    def _find_matching_logs(self, scenario: Dict) -> List[Dict]:
        """시나리오와 매칭되는 로그 찾기"""
        matched = []
        
        for log in self.logs:
            params = log['params']
            
            # 동적 고유키로 매칭
            is_match = True
            for key in self.unique_keys:
                if params.get(key, '') != scenario.get(key, ''):
                    is_match = False
                    break
            
            if is_match:
                matched.append(log)
        
        return matched

    def _print_summary(self, results: List[Dict]):
        """결과 요약 출력"""
        df = pd.DataFrame(results)
        
        total = len(df)
        passed = len(df[df['pass'] == 'PASS'])
        failed = len(df[df['pass'] == 'FAIL'])
        unexpected = len(df[df['pass'] == 'UNEXPECTED'])
        
        print(f"\n📊 검증 결과:")
        print(f"   전체: {total}개")
        print(f"   통과: {passed}개")
        print(f"   실패: {failed}개")
        print(f"   예상외: {unexpected}개")
        print(f"   성공률: {passed/total*100:.1f}%" if total > 0 else "   성공률: 0%")


# 사용 예시
def run_example():
    import datetime as dt
    timestamp = dt.datetime.today()
    formatted = timestamp.strftime("%Y-%m-%d_%H%M%S")
    
    """사용 예시"""
    analyzer = LogAnalyzer()

    # 로그 분석 및 파일 생성
    analyzer.parse_log_file_from_path(
        file_path="log_file.txt",
        output_txt=f"./result/log_analysis_result_{formatted}.txt",
        output_xlsx=f"./result/log_analysis_result_{formatted}.xlsx"
    )

    analyzer.parse_log_file_from_path(
        file_path="log_file.txt",
        output_txt=f"./result/log_analysis_result.txt",
        output_xlsx=f"./result/log_analysis_result.xlsx"
    )
if __name__ == "__main__":
    print("=== 로그 분석 및 파일 출력 ===")
    run_example()

=== 로그 분석 및 파일 출력 ===
✅ 강제 JSON 수정 성공 (라인 1)
✅ 강제 JSON 수정 성공 (라인 2)
⚠️ JSON 파싱 오류 발생: 2개
✅ 수정 성공: 2개
✅ 최종 파싱된 로그: 3개
=== 총 3개의 로그 처리 ===

✅ txt 파일 저장: ./result/log_analysis_result_2025-06-04_191629.txt
✅ Excel 파일 저장: ./result/log_analysis_result_2025-06-04_191629.xlsx
   총 3개 로그 처리됨
✅ 강제 JSON 수정 성공 (라인 1)
✅ 강제 JSON 수정 성공 (라인 2)
⚠️ JSON 파싱 오류 발생: 2개
✅ 수정 성공: 2개
✅ 최종 파싱된 로그: 3개
=== 총 3개의 로그 처리 ===

✅ txt 파일 저장: ./result/log_analysis_result.txt
✅ Excel 파일 저장: ./result/log_analysis_result.xlsx
   총 3개 로그 처리됨


In [ ]:
import pandas as pd
import json
from typing import List, Dict
import re
from urllib.parse import urlparse, parse_qs, unquote

class LogToDetailedConverter:
    """JSON 로그를 다양한 Excel 형태로 변환하는 클래스"""
    def __init__(self):
        self.input_data = []
        self.detailed_data = []
        self.detailed_deduplicated_data = []
        self.combined_data = []
        self.deduplicated_data = []

    def convert_log_to_all_formats(self, input_log_path: str, 
                                 detailed_output_path: str, 
                                 detailed_deduplicated_output_path: str,
                                 combined_output_path: str,
                                 combined_deduplicated_output_path: str):
        """
        JSON 로그 파일을 상세 형태와 결합 형태 두 가지로 변환
        
        Args:
            input_log_path: log_file.txt 파일 경로
            detailed_output_path: log_analysis_detailed.xlsx 출력 파일 경로 (각 key-value가 개별 행)
            combined_output_path: log_analysis_combined.xlsx 출력 파일 경로 (key, value가 쉼표로 구분된 열)
        """
        try:
            # 1. 입력 로그 파일 읽기
            self._load_log_file(input_log_path)
            
            # 2. 상세 데이터로 변환
            self._convert_to_detailed()
            
            # 3. 결합 데이터로 변환
            self._convert_to_combined()
            
            # 4. 결합 형태 중복 제거
            self._convert_to_combined_deduplicated()
            
            # 5. 상세 형태 중복 제거
            self._convert_to_detailed_deduplicated()
            
            # 6. 네 형태 모두 저장
            self._save_detailed_excel(detailed_output_path)
            self._save_detailed_deduplicated_excel(detailed_deduplicated_output_path)
            self._save_combined_excel(combined_output_path)
            self._save_combined_deduplicated_excel(combined_deduplicated_output_path)
            
        except Exception as e:
            print(f"❌ 변환 중 오류 발생: {e}")

    def convert_log_to_both_formats(self, input_log_path: str, 
                                  detailed_output_path: str, 
                                  combined_output_path: str):
        """
        기존 호환성을 위한 메서드 - 중복 제거 없이 두 가지 형태만 변환
        
        Args:
            input_log_path: log_file.txt 파일 경로
            detailed_output_path: log_analysis_detailed.xlsx 출력 파일 경로 (각 key-value가 개별 행)
            combined_output_path: log_analysis_combined.xlsx 출력 파일 경로 (key, value가 쉼표로 구분된 열)
        """
        try:
            # 1. 입력 로그 파일 읽기
            self._load_log_file(input_log_path)
            
            # 2. 상세 데이터로 변환
            self._convert_to_detailed()
            
            # 3. 결합 데이터로 변환
            self._convert_to_combined()
            
            # 4. 두 형태 저장
            self._save_detailed_excel(detailed_output_path)
            self._save_combined_excel(combined_output_path)
            
        except Exception as e:
            print(f"❌ 변환 중 오류 발생: {e}")

    def _load_log_file(self, log_path: str):
        """JSON 로그 파일 로드 및 파싱 (오류 자동 수정 기능 포함)"""
        try:
            self.input_data = []
            json_errors = []
            fixed_count = 0
            
            with open(log_path, 'r', encoding='utf-8') as file:
                for line_no, line in enumerate(file, 1):
                    line = line.strip()
                    if not line:
                        continue
                    
                    try:
                        # JSON 파싱
                        log_entry = json.loads(line)
                        
                        # REQUEST URL에서 파라미터 추출
                        request_url = log_entry.get('REQUEST', '')
                        parsed_data = self._parse_request_url(request_url)
                        
                        if parsed_data:
                            # 기본 로그 정보 추가
                            parsed_data['log_line'] = line_no
                            parsed_data['logtime'] = log_entry.get('LOGTIME', '')
                            parsed_data['clientip'] = log_entry.get('CLIENTIP', '')
                            parsed_data['useragent'] = log_entry.get('USERAGENT', '')
                            parsed_data['status'] = log_entry.get('STATUS', '')
                            
                            self.input_data.append(parsed_data)
                    
                    except json.JSONDecodeError as e:
                        json_errors.append((line_no, str(e), line))
                        
                        # JSON 수정 시도
                        fixed_line = self._try_fix_json(line)
                        if fixed_line:
                            try:
                                log_entry = json.loads(fixed_line)
                                
                                # REQUEST URL에서 파라미터 추출
                                request_url = log_entry.get('REQUEST', '')
                                parsed_data = self._parse_request_url(request_url)
                                
                                if parsed_data:
                                    # 기본 로그 정보 추가
                                    parsed_data['log_line'] = line_no
                                    parsed_data['logtime'] = log_entry.get('LOGTIME', '')
                                    parsed_data['clientip'] = log_entry.get('CLIENTIP', '')
                                    parsed_data['useragent'] = log_entry.get('USERAGENT', '')
                                    parsed_data['status'] = log_entry.get('STATUS', '')
                                    
                                    self.input_data.append(parsed_data)
                                    fixed_count += 1
                                    print(f"✅ JSON 수정 성공 (라인 {line_no})")
                            except:
                                # 추가 수정 시도
                                fixed_line2 = self._force_fix_json(line)
                                if fixed_line2:
                                    try:
                                        log_entry = json.loads(fixed_line2)
                                        
                                        # REQUEST URL에서 파라미터 추출
                                        request_url = log_entry.get('REQUEST', '')
                                        parsed_data = self._parse_request_url(request_url)
                                        
                                        if parsed_data:
                                            # 기본 로그 정보 추가
                                            parsed_data['log_line'] = line_no
                                            parsed_data['logtime'] = log_entry.get('LOGTIME', '')
                                            parsed_data['clientip'] = log_entry.get('CLIENTIP', '')
                                            parsed_data['useragent'] = log_entry.get('USERAGENT', '')
                                            parsed_data['status'] = log_entry.get('STATUS', '')
                                            
                                            self.input_data.append(parsed_data)
                                            fixed_count += 1
                                            print(f"✅ 강제 JSON 수정 성공 (라인 {line_no})")
                                    except:
                                        print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
                                else:
                                    print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
                        else:
                            print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
            
            # 오류 요약 출력
            if json_errors:
                print(f"⚠️ JSON 파싱 오류 발생: {len(json_errors)}개")
                if fixed_count > 0:
                    print(f"✅ 수정 성공: {fixed_count}개")
                print(f"✅ 최종 파싱된 로그: {len(self.input_data)}개")
            else:
                print(f"✅ 모든 JSON 파싱 성공")
            
            print(f"✅ 로그 파일 로드 완료: {len(self.input_data)}개 로그")
            
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {log_path}")
            raise
        except Exception as e:
            print(f"❌ 파일 로드 오류: {e}")
            raise

    def _try_fix_json(self, line: str) -> str:
        """JSON 수정 시도 - 1차 수정"""
        try:
            # 1. 마지막 쉼표 제거
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # 2. 값 내부의 따옴표를 작은따옴표로 변경
            import re
            
            # 모든 "key":"value with quotes" 패턴을 찾아서 수정
            def fix_quotes_in_value(match):
                key = match.group(1)
                value = match.group(2)
                # 값 내부의 모든 따옴표를 작은따옴표로 변경
                fixed_value = value.replace('"', "'")
                return f'"{key}":"{fixed_value}"'
            
            # 값에 따옴표가 포함된 패턴 찾기
            problem_pattern = r'"([^"]+)":"([^"]*"[^"]*)"'
            
            # 여러 번 적용하여 모든 문제 해결
            prev_line = ""
            iterations = 0
            while line != prev_line and iterations < 5:
                prev_line = line
                line = re.sub(problem_pattern, fix_quotes_in_value, line)
                iterations += 1
            
            return line
            
        except Exception:
            return None

    def _force_fix_json(self, line: str) -> str:
        """JSON 강제 수정 - 2차 수정"""
        try:
            # 기본 정리
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # JSON 구조 분석하여 키-값 쌍별로 처리
            if not (line.startswith('{') and line.endswith('}')):
                return None
            
            # 중괄호 제거
            content = line[1:-1]
            
            # 키-값 쌍들을 분리
            pairs = []
            current_pair = ""
            bracket_count = 0
            in_quotes = False
            escape_next = False
            
            for char in content:
                if escape_next:
                    current_pair += char
                    escape_next = False
                    continue
                
                if char == '\\':
                    current_pair += char
                    escape_next = True
                    continue
                
                if char == '"' and not escape_next:
                    in_quotes = not in_quotes
                
                if not in_quotes:
                    if char in '{}[]':
                        bracket_count += 1 if char in '{[' else -1
                    elif char == ',' and bracket_count == 0:
                        pairs.append(current_pair.strip())
                        current_pair = ""
                        continue
                
                current_pair += char
            
            if current_pair.strip():
                pairs.append(current_pair.strip())
            
            # 각 키-값 쌍을 수정
            fixed_pairs = []
            for pair in pairs:
                if ':' in pair:
                    try:
                        # 키와 값 분리 (첫 번째 콜론 기준)
                        colon_pos = pair.find(':')
                        key_part = pair[:colon_pos].strip()
                        value_part = pair[colon_pos+1:].strip()
                        
                        # 값 부분에서 따옴표로 둘러싸인 문자열인지 확인
                        if value_part.startswith('"') and value_part.endswith('"'):
                            # 값 내부의 따옴표를 작은따옴표로 변경
                            inner_value = value_part[1:-1]  # 양쪽 따옴표 제거
                            fixed_inner = inner_value.replace('"', "'")
                            fixed_value = f'"{fixed_inner}"'
                            fixed_pairs.append(f'{key_part}:{fixed_value}')
                        else:
                            fixed_pairs.append(pair)
                    except:
                        fixed_pairs.append(pair)
                else:
                    fixed_pairs.append(pair)
            
            # 다시 조합
            result = '{' + ','.join(fixed_pairs) + '}'
            return result
            
        except Exception:
            return None

    def _parse_request_url(self, request_url: str) -> Dict:
        """REQUEST URL에서 파라미터들을 추출 (개선된 한글 처리)"""
        if not request_url:
            return {}
        
        try:
            # 유니코드 이스케이프 디코딩
            url = request_url.encode('utf-8').decode('unicode_escape')
            parsed_url = urlparse(url)
            
            # 쿼리 파라미터를 수동으로 파싱 (복잡한 값 처리)
            query = parsed_url.query
            processed_params = {}
            
            if not query:
                return processed_params
            
            # &로 분할하되, URL 인코딩된 &(%26)은 분할하지 않음
            params = []
            current_param = ""
            i = 0
            
            while i < len(query):
                if query[i:i+1] == '&' and not query[max(0, i-3):i] == '%26':
                    if current_param:
                        params.append(current_param)
                    current_param = ""
                else:
                    current_param += query[i]
                i += 1
            
            if current_param:
                params.append(current_param)
            
            # 각 파라미터를 키=값으로 분할
            for param in params:
                if '=' not in param:
                    continue
                    
                # 첫 번째 = 기준으로만 분할 (값에 =이 포함될 수 있음)
                key, value = param.split('=', 1)
                
                # 키와 값 디코딩
                decoded_key = self._fix_korean_text(key)
                decoded_value = self._fix_korean_text(value)
                
                processed_params[decoded_key] = decoded_value
            
            return processed_params
            
        except Exception as e:
            print(f"URL 파싱 오류: {e}")
            # 실패 시 기존 방식으로 fallback
            try:
                url = request_url.encode('utf-8').decode('unicode_escape')
                parsed_url = urlparse(url)
                query_params = parse_qs(parsed_url.query, keep_blank_values=True)
                
                processed_params = {}
                for key, value_list in query_params.items():
                    raw_value = value_list[0] if value_list else ''
                    decoded_value = self._fix_korean_text(raw_value)
                    processed_params[key] = decoded_value
                
                return processed_params
            except:
                return {}

    def _fix_korean_text(self, text: str) -> str:
        """깨진 한글을 올바르게 디코딩"""
        try:
            # URL 디코딩 먼저 시도
            from urllib.parse import unquote
            decoded = unquote(text, encoding='utf-8')
            
            # 깨진 한글 패턴이 있는지 확인
            if self._has_broken_korean(decoded):
                # Latin-1로 인코딩 후 UTF-8로 디코딩 (일반적인 mojibake 해결법)
                try:
                    fixed = decoded.encode('latin-1').decode('utf-8')
                    return fixed
                except:
                    pass
                    
                # 그래도 안되면 바이트 단위로 처리
                try:
                    # 문자열의 각 문자를 바이트로 변환 후 UTF-8 디코딩
                    byte_data = bytes([ord(c) for c in decoded if ord(c) < 256])
                    fixed = byte_data.decode('utf-8', errors='ignore')
                    return fixed
                except:
                    pass
                    
                # UTF-8 바이트 패턴으로 복구 시도
                try:
                    # 특정 깨진 패턴들을 직접 매핑
                    broken_patterns = {
                        'ê·¹ììëª¨': '극장용모',
                        'í¥ê¸°ë¡­ê²': '향기롭게',
                        'ì¤ê¸°ì¼ì´': '수기일이',
                        'ììëª¨': '용모',
                        'ê·¹': '극'
                    }
                    
                    result = decoded
                    for broken, fixed in broken_patterns.items():
                        result = result.replace(broken, fixed)
                    
                    if result != decoded:
                        return result
                        
                except:
                    pass
            
            return decoded
            
        except Exception:
            return text

    def _has_broken_korean(self, text: str) -> bool:
        """깨진 한글 패턴이 있는지 확인"""
        broken_patterns = [
            'ë¶', 'ìë', 'ê¼¬', 'ë', 'í¼', 'ì¸', 'ê¸°', 'ê¸', 'ìì¹', 'ì', 'ê',
            'ê·¹ì', 'í¥ê¸°', 'ì¤ê¸°', 'ììë', 'ëª¨'
        ]
        return any(pattern in text for pattern in broken_patterns)

    def _convert_to_detailed(self):
        """로그 데이터를 상세 key-value 행으로 변환"""
        self.detailed_data = []
        
        # 제외할 기본 필드들 (메타데이터)
        exclude_fields = {'log_line', 'logtime', 'clientip', 'useragent', 'status'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 기본 정보 추출
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')  # click_text 추가
            
            # 나머지 파라미터들을 key-value 쌍으로 처리
            for key, value in log_entry.items():
                if key in exclude_fields:
                    continue
                
                # 기본 필드들은 건너뛰기 (click_text는 이제 포함됨)
                if key in ['page_id', 'click_type', 'act_type']:
                    continue
                
                self.detailed_data.append({
                    'no': log_no,
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'click_text': click_text,
                    'key': key,
                    'value': str(value) if value is not None else ''
                })
        
        print(f"✅ 상세 데이터 변환 완료: {len(self.detailed_data)}개 key-value 쌍")

    def _convert_to_combined(self):
        """로그 데이터를 결합 형태로 변환 (key, value가 쉼표로 구분된 열)"""
        self.combined_data = []
        
        # 제외할 기본 필드들 (메타데이터)
        exclude_fields = {'log_line', 'logtime', 'clientip', 'useragent', 'status', 
                         'page_id', 'click_type', 'act_type'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 기본 정보 추출
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')  # click_text 추가
            
            # 나머지 파라미터들 수집 (이제 click_text도 포함됨)
            keys = []
            values = []
            
            for key, value in log_entry.items():
                if key not in exclude_fields:
                    keys.append(key)
                    values.append(str(value) if value is not None else '')
            
            # 키와 값을 쉼표로 구분된 문자열로 결합
            keys_combined = ', '.join(keys) if keys else ''
            values_combined = ', '.join(values) if values else ''
            
            combined_entry = {
                'no': log_no,
                'page_id': page_id,
                'click_type': click_type,
                'act_type': act_type,
                'keys_combined': keys_combined,
                'values_combined': values_combined,
                'key_count': len(keys)
            }
            
            # click_text가 있으면 추가
            if click_text:
                combined_entry['click_text'] = click_text
            
            self.combined_data.append(combined_entry)
        
        print(f"✅ 결합 데이터 변환 완료: {len(self.combined_data)}개 로그")

    def _convert_to_combined_deduplicated(self):
        """결합 데이터에서 중복을 제거한 형태로 변환"""
        if not self.combined_data:
            print("⚠️ 결합 데이터가 없어 중복 제거를 수행할 수 없습니다.")
            return
        
        # pandas DataFrame으로 변환
        df = pd.DataFrame(self.combined_data)
        
        # 중복 제거 전 개수
        original_count = len(df)
        
        # page_id, click_type, act_type을 기준으로 중복 제거
        # click_text가 있는 경우 추가로 포함
        dedup_columns = ['page_id', 'click_type', 'act_type']
        
        # click_text 컬럼이 존재하고 비어있지 않은 행이 있으면 추가
        if 'click_text' in df.columns and df['click_text'].notna().any() and (df['click_text'] != '').any():
            dedup_columns.append('click_text')
        
        # 중복 제거
        df_deduplicated = df.drop_duplicates(subset=dedup_columns, keep='first')
        
        # 중복 제거 후 개수
        deduplicated_count = len(df_deduplicated)
        removed_count = original_count - deduplicated_count
        
        # DataFrame을 다시 딕셔너리 리스트로 변환
        self.deduplicated_data = df_deduplicated.to_dict('records')
        
        print(f"✅ 결합 형태 중복 제거 완료:")
        print(f"   원본: {original_count}개")
        print(f"   중복 제거 후: {deduplicated_count}개")
        print(f"   제거된 중복: {removed_count}개")
        print(f"   중복 제거 기준: {', '.join(dedup_columns)}")

    def _convert_to_detailed_deduplicated(self):
        """상세 데이터에서 중복을 제거한 형태로 변환"""
        if not self.detailed_data:
            print("⚠️ 상세 데이터가 없어 중복 제거를 수행할 수 없습니다.")
            return
        
        # pandas DataFrame으로 변환
        df = pd.DataFrame(self.detailed_data)
        
        # 중복 제거 전 개수
        original_count = len(df)
        
        # page_id, click_type, act_type, key, value를 기준으로 중복 제거
        # click_text가 있는 경우 추가로 포함
        dedup_columns = ['page_id', 'click_type', 'act_type', 'key', 'value']
        
        # click_text 컬럼이 존재하고 비어있지 않은 행이 있으면 추가
        if 'click_text' in df.columns and df['click_text'].notna().any() and (df['click_text'] != '').any():
            dedup_columns.append('click_text')
        
        # 중복 제거
        df_deduplicated = df.drop_duplicates(subset=dedup_columns, keep='first')
        
        # 중복 제거 후 개수
        deduplicated_count = len(df_deduplicated)
        removed_count = original_count - deduplicated_count
        
        # DataFrame을 다시 딕셔너리 리스트로 변환
        self.detailed_deduplicated_data = df_deduplicated.to_dict('records')
        
        print(f"✅ 상세 형태 중복 제거 완료:")
        print(f"   원본: {original_count}개")
        print(f"   중복 제거 후: {deduplicated_count}개")
        print(f"   제거된 중복: {removed_count}개")
        print(f"   중복 제거 기준: {', '.join(dedup_columns)}")

    def _extract_click_text_from_keys(self, row):
        """keys_combined에서 click_text 값을 추출하려고 시도"""
        try:
            keys = row.get('keys_combined', '').split(', ')
            values = row.get('values_combined', '').split(', ')
            
            if len(keys) == len(values):
                for i, key in enumerate(keys):
                    if key.strip() == 'click_text':
                        return values[i].strip() if i < len(values) else ''
            return ''
        except:
            return ''

    def _save_detailed_excel(self, output_path: str):
        """상세 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.detailed_data)
            
            if df.empty:
                print("⚠️ 저장할 상세 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의
            column_order = ['no', 'page_id', 'click_type', 'act_type', 'click_text', 'key', 'value']
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 상세 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.detailed_data)}개 key-value 쌍")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_detailed_deduplicated_excel(self, output_path: str):
        """상세 중복 제거된 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.detailed_deduplicated_data)
            
            if df.empty:
                print("⚠️ 저장할 상세 중복 제거 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의
            column_order = ['no', 'page_id', 'click_type', 'act_type', 'click_text', 'key', 'value']
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 상세 중복 제거 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.detailed_deduplicated_data)}개 key-value 쌍 (중복 제거됨)")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_combined_deduplicated_excel(self, output_path: str):
        """결합 형태 중복 제거된 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.deduplicated_data)
            
            if df.empty:
                print("⚠️ 저장할 중복 제거 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의
            column_order = ['no', 'page_id', 'click_type', 'act_type', 'keys_combined', 'values_combined', 'key_count']
            
            # click_text 컬럼이 있으면 추가
            if 'click_text' in df.columns:
                column_order.insert(4, 'click_text')
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 결합 형태 중복 제거 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.deduplicated_data)}개 로그 (중복 제거됨)")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_combined_excel(self, output_path: str):
        """결합 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.combined_data)
            
            if df.empty:
                print("⚠️ 저장할 결합 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의
            column_order = ['no', 'page_id', 'click_type', 'act_type', 'keys_combined', 'values_combined', 'key_count']
            
            # click_text 컬럼이 있으면 추가
            if 'click_text' in df.columns:
                column_order.insert(4, 'click_text')
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 결합 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.combined_data)}개 로그")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def get_conversion_summary(self) -> Dict:
        """변환 결과 요약 정보 반환"""
        if not self.detailed_data:
            return {"error": "변환된 데이터가 없습니다."}
        
        # 통계 계산
        total_logs = len(self.input_data)
        total_key_values = len(self.detailed_data)
        avg_keys_per_log = total_key_values / total_logs if total_logs > 0 else 0
        
        # 가장 많은 키를 가진 로그 찾기
        key_counts = {}
        for item in self.detailed_data:
            log_no = item['no']
            key_counts[log_no] = key_counts.get(log_no, 0) + 1
        
        max_keys = max(key_counts.values()) if key_counts else 0
        min_keys = min(key_counts.values()) if key_counts else 0
        
        # 고유한 키 수 계산
        unique_keys = set(item['key'] for item in self.detailed_data)
        
        # 중복 제거 통계
        combined_count = len(self.combined_data) if self.combined_data else 0
        combined_deduplicated_count = len(self.deduplicated_data) if self.deduplicated_data else 0
        combined_duplicates_removed = combined_count - combined_deduplicated_count
        
        detailed_count = len(self.detailed_data) if self.detailed_data else 0
        detailed_deduplicated_count = len(self.detailed_deduplicated_data) if self.detailed_deduplicated_data else 0
        detailed_duplicates_removed = detailed_count - detailed_deduplicated_count
        
        return {
            "total_logs": total_logs,
            "total_key_value_pairs": total_key_values,
            "average_keys_per_log": round(avg_keys_per_log, 2),
            "max_keys_in_single_log": max_keys,
            "min_keys_in_single_log": min_keys,
            "unique_keys_count": len(unique_keys),
            "unique_keys": sorted(list(unique_keys)),
            "combined_logs_count": combined_count,
            "combined_deduplicated_count": combined_deduplicated_count,
            "combined_duplicates_removed": combined_duplicates_removed,
            "detailed_key_value_pairs": detailed_count,
            "detailed_deduplicated_count": detailed_deduplicated_count,
            "detailed_duplicates_removed": detailed_duplicates_removed
        }

    def preview_data(self, num_rows: int = 10):
        """변환된 상세 데이터 미리보기"""
        print(f"\n📋 상세 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 140)
        print(f"{'No':3} | {'Page ID':25} | {'Click Type':12} | {'Act Type':12} | {'Click Text':15} | {'Key':20} | {'Value':25}")
        print("-" * 140)
        
        if self.detailed_data:
            for item in self.detailed_data[:num_rows]:
                page_id = item['page_id'][:25] if len(item['page_id']) > 25 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                click_text = str(item.get('click_text', ''))[:15] if len(str(item.get('click_text', ''))) > 15 else str(item.get('click_text', ''))
                key = item['key'][:20] if len(item['key']) > 20 else item['key']
                value = item['value'][:25] if len(item['value']) > 25 else item['value']
                
                print(f"{item['no']:3} | {page_id:25} | {click_type:12} | {act_type:12} | {click_text:15} | {key:20} | {value:25}")
            
            if len(self.detailed_data) > num_rows:
                print(f"... 외 {len(self.detailed_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_deduplicated_data(self, num_rows: int = 5):
        """결합 형태 중복 제거된 데이터 미리보기"""
        print(f"\n📋 결합 형태 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 160)
        
        # 헤더 동적 생성
        has_click_text = self.deduplicated_data and any('click_text' in item for item in self.deduplicated_data[:3])
        if has_click_text:
            print(f"{'No':3} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Click Text':12} | {'Keys':25} | {'Values':25} | {'Count':5}")
        else:
            print(f"{'No':3} | {'Page ID':25} | {'Click Type':12} | {'Act Type':12} | {'Keys':30} | {'Values':30} | {'Count':5}")
        
        print("-" * 160)
        
        if self.deduplicated_data:
            for item in self.deduplicated_data[:num_rows]:
                page_id = item['page_id'][:20] if len(item['page_id']) > 20 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                
                if 'click_text' in item and item['click_text']:
                    click_text = item['click_text'][:12] if len(str(item['click_text'])) > 12 else str(item['click_text'])
                    keys = item['keys_combined'][:25] if len(item['keys_combined']) > 25 else item['keys_combined']
                    values = item['values_combined'][:25] if len(item['values_combined']) > 25 else item['values_combined']
                    print(f"{item['no']:3} | {page_id:20} | {click_type:12} | {act_type:12} | {click_text:12} | {keys:25} | {values:25} | {item['key_count']:5}")
                else:
                    keys = item['keys_combined'][:25] if len(item['keys_combined']) > 25 else item['keys_combined']
                    values = item['values_combined'][:25] if len(item['values_combined']) > 25 else item['values_combined']
                    print(f"{item['no']:3} | {page_id:25} | {click_type:12} | {act_type:12} | {keys:25} | {values:25} | {item['key_count']:5}")
            
            if len(self.deduplicated_data) > num_rows:
                print(f"... 외 {len(self.deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_detailed_deduplicated_data(self, num_rows: int = 10):
        """상세 중복 제거된 데이터 미리보기"""
        print(f"\n📋 상세 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 140)
        print(f"{'No':3} | {'Page ID':25} | {'Click Type':12} | {'Act Type':12} | {'Click Text':15} | {'Key':20} | {'Value':25}")
        print("-" * 140)
        
        if self.detailed_deduplicated_data:
            for item in self.detailed_deduplicated_data[:num_rows]:
                page_id = item['page_id'][:25] if len(item['page_id']) > 25 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                click_text = str(item.get('click_text', ''))[:15] if len(str(item.get('click_text', ''))) > 15 else str(item.get('click_text', ''))
                key = item['key'][:20] if len(item['key']) > 20 else item['key']
                value = item['value'][:25] if len(item['value']) > 25 else item['value']
                
                print(f"{item['no']:3} | {page_id:25} | {click_type:12} | {act_type:12} | {click_text:15} | {key:20} | {value:25}")
            
            if len(self.detailed_deduplicated_data) > num_rows:
                print(f"... 외 {len(self.detailed_deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_deduplicated_data(self, num_rows: int = 5):
        """결합 형태 중복 제거된 데이터 미리보기"""
        print(f"\n📋 중복 제거된 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 160)
        
        # 헤더 동적 생성
        has_click_text = self.deduplicated_data and any('click_text' in item for item in self.deduplicated_data[:3])
        if has_click_text:
            print(f"{'No':3} | {'Page ID':20} | {'Click Type':12} | {'Act Type':12} | {'Click Text':12} | {'Keys':25} | {'Values':25} | {'Count':5}")
        else:
            print(f"{'No':3} | {'Page ID':25} | {'Click Type':12} | {'Act Type':12} | {'Keys':30} | {'Values':30} | {'Count':5}")
        
        print("-" * 160)
        
        if self.deduplicated_data:
            for item in self.deduplicated_data[:num_rows]:
                page_id = item['page_id'][:20] if len(item['page_id']) > 20 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                
                if 'click_text' in item and item['click_text']:
                    click_text = item['click_text'][:12] if len(str(item['click_text'])) > 12 else str(item['click_text'])
                    keys = item['keys_combined'][:25] if len(item['keys_combined']) > 25 else item['keys_combined']
                    values = item['values_combined'][:25] if len(item['values_combined']) > 25 else item['values_combined']
                    print(f"{item['no']:3} | {page_id:20} | {click_type:12} | {act_type:12} | {click_text:12} | {keys:25} | {values:25} | {item['key_count']:5}")
                else:
                    keys = item['keys_combined'][:25] if len(item['keys_combined']) > 25 else item['keys_combined']
                    values = item['values_combined'][:25] if len(item['values_combined']) > 25 else item['values_combined']
                    print(f"{item['no']:3} | {page_id:25} | {click_type:12} | {act_type:12} | {keys:25} | {values:25} | {item['key_count']:5}")
            
            if len(self.deduplicated_data) > num_rows:
                print(f"... 외 {len(self.deduplicated_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def preview_combined_data(self, num_rows: int = 5):
        """결합 데이터 미리보기"""
        print(f"\n📋 결합 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 150)
        print(f"{'No':3} | {'Page ID':25} | {'Click Type':12} | {'Act Type':12} | {'Keys':40} | {'Values':40} | {'Count':5}")
        print("-" * 150)
        
        if self.combined_data:
            for item in self.combined_data[:num_rows]:
                page_id = item['page_id'][:25] if len(item['page_id']) > 25 else item['page_id']
                click_type = item['click_type'][:12] if len(item['click_type']) > 12 else item['click_type']
                act_type = item['act_type'][:12] if len(item['act_type']) > 12 else item['act_type']
                keys = item['keys_combined'][:40] if len(item['keys_combined']) > 40 else item['keys_combined']
                values = item['values_combined'][:40] if len(item['values_combined']) > 40 else item['values_combined']
                
                print(f"{item['no']:3} | {page_id:25} | {click_type:12} | {act_type:12} | {keys:40} | {values:40} | {item['key_count']:5}")
            
            if len(self.combined_data) > num_rows:
                print(f"... 외 {len(self.combined_data) - num_rows}개 행")
        else:
            print("변환된 데이터가 없습니다.")

    def get_unique_keys_analysis(self):
        """고유 키 분석"""
        if not self.detailed_data:
            print("분석할 데이터가 없습니다.")
            return
        
        # 키별 빈도수 계산
        key_frequency = {}
        for item in self.detailed_data:
            key = item['key']
            key_frequency[key] = key_frequency.get(key, 0) + 1
        
        # 빈도수 순으로 정렬
        sorted_keys = sorted(key_frequency.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\n📊 키 분석 (총 {len(sorted_keys)}개 고유 키):")
        print("-" * 50)
        print(f"{'Key':30} | {'빈도수':10}")
        print("-" * 50)
        
        for key, freq in sorted_keys[:20]:  # 상위 20개만 표시
            key_display = key[:30] if len(key) > 30 else key
            print(f"{key_display:30} | {freq:10}")
        
        if len(sorted_keys) > 20:
            print(f"... 외 {len(sorted_keys) - 20}개 키")


def run_log_converter_example():
    """로그 변환기 사용 예시"""
    converter = LogToDetailedConverter()

    print("=== JSON 로그를 Excel 형태로 변환 (JSON 오류 자동 수정 + 중복 제거 기능 포함) ===\n")
    
    # 로그 파일을 네 가지 형태로 변환
    converter.convert_log_to_all_formats(
        "log_file_.txt",                              # 입력 JSON 로그 파일
        "./result/log_analysis_detailed.xlsx",                # 상세 형태 출력
        "./result/log_analysis_detailed_deduplicated.xlsx",   # 상세 형태 중복 제거 출력
        "./result/log_analysis_combined.xlsx",                # 결합 형태 출력
        "./result/log_analysis_combined_deduplicated.xlsx"    # 결합 형태 중복 제거 출력
    )
    
    # 결과 요약 출력
    summary = converter.get_conversion_summary()
    print(f"\n📊 변환 결과 요약:")
    for key, value in summary.items():
        if key != 'unique_keys':  # unique_keys는 너무 길어서 별도 출력
            print(f"   {key}: {value}")
    
    # 데이터 미리보기
    converter.preview_data(10)
    converter.preview_detailed_deduplicated_data(10)
    converter.preview_combined_data(5)
    converter.preview_combined_deduplicated_data(5)
    
    # 키 분석
    converter.get_unique_keys_analysis()


if __name__ == "__main__":
    run_log_converter_example()

=== JSON 로그를 Excel 형태로 변환 (JSON 오류 자동 수정 + 중복 제거 기능 포함) ===

✅ 강제 JSON 수정 성공 (라인 94)
✅ 강제 JSON 수정 성공 (라인 422)
✅ 강제 JSON 수정 성공 (라인 464)
✅ 강제 JSON 수정 성공 (라인 710)
✅ 강제 JSON 수정 성공 (라인 731)
✅ 강제 JSON 수정 성공 (라인 738)
✅ 강제 JSON 수정 성공 (라인 777)
⚠️ JSON 파싱 오류 발생: 7개
✅ 수정 성공: 7개
✅ 최종 파싱된 로그: 777개
✅ 로그 파일 로드 완료: 777개 로그
✅ 상세 데이터 변환 완료: 4495개 key-value 쌍
✅ 결합 데이터 변환 완료: 777개 로그
✅ 결합 형태 중복 제거 완료:
   원본: 777개
   중복 제거 후: 297개
   제거된 중복: 480개
   중복 제거 기준: page_id, click_type, act_type, click_text
✅ 상세 형태 중복 제거 완료:
   원본: 4495개
   중복 제거 후: 2260개
   제거된 중복: 2235개
   중복 제거 기준: page_id, click_type, act_type, key, value, click_text
✅ 상세 Excel 파일 저장: ./result/log_analysis_detailed.xlsx
   총 4495개 key-value 쌍
✅ 상세 중복 제거 Excel 파일 저장: ./result/log_analysis_detailed_deduplicated.xlsx
   총 2260개 key-value 쌍 (중복 제거됨)
✅ 결합 Excel 파일 저장: ./result/log_analysis_combined.xlsx
   총 777개 로그
✅ 결합 형태 중복 제거 Excel 파일 저장: ./result/log_analysis_combined_deduplicated.xlsx
   총 297개 로그 (중복 제거됨)

📊 변환 결과 요약:
   total_logs: 777
   